We will begin by setting out Imports and reading in our dataset 

In [18]:
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt

from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split , cross_val_score
from sklearn.linear_model import LinearRegression , LassoCV , RidgeCV

In [19]:
df = pd.read_csv('../datasets/test.csv')

In [20]:
# lets make the names pythonic
df.columns = df.columns.str.replace(" ","_")

In [21]:
df.columns

Index(['Id', 'PID', 'MS_SubClass', 'MS_Zoning', 'Lot_Frontage', 'Lot_Area',
       'Street', 'Alley', 'Lot_Shape', 'Land_Contour', 'Utilities',
       'Lot_Config', 'Land_Slope', 'Neighborhood', 'Condition_1',
       'Condition_2', 'Bldg_Type', 'House_Style', 'Overall_Qual',
       'Overall_Cond', 'Year_Built', 'Year_Remod/Add', 'Roof_Style',
       'Roof_Matl', 'Exterior_1st', 'Exterior_2nd', 'Mas_Vnr_Type',
       'Mas_Vnr_Area', 'Exter_Qual', 'Exter_Cond', 'Foundation', 'Bsmt_Qual',
       'Bsmt_Cond', 'Bsmt_Exposure', 'BsmtFin_Type_1', 'BsmtFin_SF_1',
       'BsmtFin_Type_2', 'BsmtFin_SF_2', 'Bsmt_Unf_SF', 'Total_Bsmt_SF',
       'Heating', 'Heating_QC', 'Central_Air', 'Electrical', '1st_Flr_SF',
       '2nd_Flr_SF', 'Low_Qual_Fin_SF', 'Gr_Liv_Area', 'Bsmt_Full_Bath',
       'Bsmt_Half_Bath', 'Full_Bath', 'Half_Bath', 'Bedroom_AbvGr',
       'Kitchen_AbvGr', 'Kitchen_Qual', 'TotRms_AbvGrd', 'Functional',
       'Fireplaces', 'Fireplace_Qu', 'Garage_Type', 'Garage_Yr_Blt',
       'G

# CLEANING

Looking at the size of our dataset we elected to narrow the feature our model incorporates down to those we feel will be most predictive. We can come back here latter to add new feature from our original DF to our working copy in the new_df.

In [22]:
# Dataframe of features we selected
new_df = df[['Lot_Area','1st_Flr_SF','2nd_Flr_SF',
             'Total_Bsmt_SF',
             'Central_Air','Id','Garage_Area',
             'BsmtFin_SF_2','BsmtFin_SF_1']].copy()

#Set index to house ID for submission format
new_df.set_index('Id', inplace = True)

Now we need to Create a column for Sq_ft from the 3 columns we have pertaining to that value.

In [23]:
# Creating a feature for Total Sq feet 

new_df['Sq_ft'] = new_df['1st_Flr_SF'] + new_df['2nd_Flr_SF'] + new_df['Total_Bsmt_SF']

In [24]:
# Dropping the Original columns used to make Sq_ft

new_df.drop(columns = ['1st_Flr_SF','2nd_Flr_SF','Total_Bsmt_SF'], inplace = True)
new_df.head()

,Lot_Area,Central_Air,Garage_Area,BsmtFin_SF_2,BsmtFin_SF_1,Sq_ft
Id,,,,,,
2658,9142,N,440,0,0,2948
2718,9662,Y,580,0,0,3934
2414,17104,Y,426,0,554,2150
1989,8520,Y,480,0,0,1936
625,9500,Y,514,0,609,2788


In [25]:
# DO NOT RUN ME TWICE


# Making our HasCentral Air a binary column 
new_df.Central_Air = np.where(new_df.Central_Air == 'Y',1,0)


In [26]:
# ADD NEW DROPS OR FILTERS HERE 






# Features

## Has_Garage

Has Garage True/False

In [27]:
# Lets make a column for Has_Garage
new_df['Has_Garage'] = np.where(new_df.Garage_Area > 0.0 , 1, 0)

# and drop the garage Sq ft column
new_df.drop(columns = ['Garage_Area'], inplace = True)

## Large_Lot

In [28]:
new_df['Large_Lot'] = np.where(new_df['Lot_Area'] > 12_000, 1, 0)

## Finished_Basement

In [29]:
# one for houses with finished basements
# first lets make a column for whether or not a home has a finsihed basement

# we need to ass the two measure of finished basements together 
new_df['fin_base_sqft'] =  new_df['BsmtFin_SF_2'] + new_df['BsmtFin_SF_1']

# make column for if basement if fisnished 
new_df['Finished_Basement'] = np.where(new_df['fin_base_sqft'] > 0.0 , 1,0)

In [30]:
# Drop the two measures of finished basement
new_df.drop(columns = ['BsmtFin_SF_2','BsmtFin_SF_1','fin_base_sqft'],inplace = True)

# Export

In [31]:
new_df.head()

,Lot_Area,Central_Air,Sq_ft,Has_Garage,Large_Lot,Finished_Basement
Id,,,,,,
2658,9142,0,2948,1,0,0
2718,9662,1,3934,1,0,0
2414,17104,1,2150,1,1,1
1989,8520,1,1936,1,0,0
625,9500,1,2788,1,0,1


In [32]:
# we still need to add the house id # as index 
new_df.to_csv('../datasets/Clean_Test.csv')

In [33]:
new_df.shape

(878, 6)